In [43]:
using JSON, DataFrames, CSV, DataFramesMeta, Plots, Dates, HTTP, Printf, Statistics

In [11]:
const extended = "MyDataExtended"
const current = "MyDataCurrent"
const technical = "TechnicalData"
const features = "track_features"
const SpotifyApiURL = "https://api.spotify.com/v1/tracks/";

### Track features dataset

In [18]:
# feats = vcat([DataFrame(CSV.File(joinpath(features, file))) for file in readdir(features) if !occursin("._", file)]...)

In [20]:
df = DataFrame(vcat([JSON.parsefile(joinpath(extended, file)) for file in readdir(extended) if occursin("endsong", file) ]...))
df.ts = DateTime.(df.ts, "yyyy-m-dTH:M:SZ");

In [21]:
x = split(first(df.spotify_track_uri), ":")[end]

"7iAqvWLgZzXvH38lA06QZg"

In [53]:
sort(df, :ts)

Row,offline_timestamp,conn_country,master_metadata_album_album_name,username,spotify_track_uri,reason_start,ts,master_metadata_album_artist_name,offline,spotify_episode_uri,platform,reason_end,user_agent_decrypted,ms_played,master_metadata_track_name,ip_addr_decrypted,skipped,incognito_mode,episode_show_name,episode_name,shuffle
,Int64,String,Union…,String,Union…,String,DateTime,Union…,Bool,Union…,String,String,Union…,Int64,Union…,String,Union…,Bool,Union…,Union…,Bool
1,0,AR,Light and Heavy/The Best of...,11140152173,spotify:track:41TqRBgF4Ahyr4vGmZVk8b,appload,2014-05-11T23:58:43,Iron Butterfly,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,15629,In-A-Gadda-Da-Vida - Single Version,186.137.52.244,true,false,,,false
2,0,AR,Presente,11140152173,spotify:track:03PfjjVBo7pX8AOzfCJOhM,appload,2014-05-11T23:59:30,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,34080,Pena en mi corazon,186.137.52.244,true,false,,,false
3,0,AR,Presente,11140152173,spotify:track:3DBr8IWWZhH2F9KHmcBWN1,appload,2014-05-12T00:16:26,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",endplay,unknown,5572,La trufa y el sifon,186.137.52.244,true,false,,,false
4,0,AR,Presente,11140152173,spotify:track:2AtJX42EiltEH5FDTFYFyb,appload,2014-05-12T00:21:09,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,0,Caminante,186.137.52.244,true,false,,,false
5,0,AR,Presente,11140152173,spotify:track:5O4LBJJWwaso2p32Oz2q8Q,appload,2014-05-12T00:21:10,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,0,Pide piso,186.137.52.244,true,false,,,false
6,0,AR,Presente,11140152173,spotify:track:2AtJX42EiltEH5FDTFYFyb,appload,2014-05-12T00:23:17,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",backbtn,unknown,126037,Caminante,186.137.52.244,true,false,,,false
7,0,AR,Presente,11140152173,spotify:track:03PfjjVBo7pX8AOzfCJOhM,backbtn,2014-05-12T00:23:17,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",backbtn,unknown,0,Pena en mi corazon,186.137.52.244,true,false,,,false
8,0,AR,Presente,11140152173,spotify:track:4N5n65Zs60cSFRipzejIcZ,backbtn,2014-05-12T00:23:35,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,18204,Nocturno,186.137.52.244,true,false,,,false
9,0,AR,Presente,11140152173,spotify:track:3DBr8IWWZhH2F9KHmcBWN1,appload,2014-05-12T00:30:00,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",trackdone,unknown,251813,La trufa y el sifon,186.137.52.244,false,false,,,false


In [52]:
gdf = groupby(df, :master_metadata_album_artist_name)
skips = combine(gdf, :reason_end => ( x -> (freq_skip = sum(x .== "fwdbtn")/length(x) , rawskip = sum(x .== "fwdbtn"), count = length(x)) ) => AsTable )
@transform! skips begin
    :meanskip = mean(:rawskip)
end 
@subset!(skips, :rawskip .> :meanskip)
sort(skips, [:freq_skip, :rawskip] ,rev = true)


Row,master_metadata_album_artist_name,freq_skip,rawskip,count,meanskip
,Union…,Float64,Int64,Int64,Float64
1,Deftones,0.911111,41,45,6.37387
2,Don McLean,0.854545,47,55,6.37387
3,The Flaming Lips,0.8125,13,16,6.37387
4,Nina Hagen,0.809524,17,21,6.37387
5,Studio Allstars,0.784615,51,65,6.37387
6,Primus,0.777778,105,135,6.37387
7,Bruce Springsteen,0.777778,49,63,6.37387
8,Black Sabbath,0.765957,72,94,6.37387
9,Entre Rios,0.75,51,68,6.37387


In [56]:
Year(first(df.ts)) == Year(2021)

true

In [71]:
Dates.Millisecond(sum((@subset(df, Year.(:ts) .== Year.(2022))).ms_played))

2883116839 milliseconds

In [82]:
songset = Dict(song.spotify_track_uri => Dict( :title => song.master_metadata_track_name , :artist => song.master_metadata_album_artist_name, :album => song.master_metadata_album_album_name ) for song in eachrow(unique(df, :spotify_track_uri)))

Dict{Union{Nothing, String}, Dict{Symbol}} with 12257 entries:
  "spotify:track:1XiVySbno… => Dict(:artist=>"Portishead", :title=>"Nylon Smile…
  "spotify:track:4jQSsfjzk… => Dict(:artist=>"La Gran Perdida De Energia", :tit…
  "spotify:track:0uCKawvDK… => Dict(:artist=>"El Cuarteto De Nos", :title=>"El …
  "spotify:track:6V34A7nIj… => Dict(:artist=>"Varese", :title=>"Testigo", :albu…
  "spotify:track:2bWQKPzPv… => Dict(:artist=>"Rosario Bléfari", :title=>"Viento…
  "spotify:track:39nNw4Q0w… => Dict(:artist=>"The Alan Parsons Project", :title…
  "spotify:track:26loAxh2z… => Dict(:artist=>"Dancing Mood", :title=>"The Long …
  "spotify:track:20lBNE9YA… => Dict(:artist=>"Miguel Campbell", :title=>"Someth…
  "spotify:track:1tHkKvdWA… => Dict(:artist=>"Los Tipitos", :title=>"Sólo figur…
  "spotify:track:0ax2Np3bX… => Dict(:artist=>"Parcels", :title=>"Overnight - fr…
  "spotify:track:4vv16N4cy… => Dict(:artist=>"The Who", :title=>"Join Together"…
  "spotify:track:053qYbsCw… => Dict(:artist=>"

Dict{Pair{Symbol}, Pair{Symbol}} with 10716 entries:
  :title=>"The Ocean - Live"                     => :artist=>"Led Zeppelin"
  :title=>"Ayer Me Llamó Mi Ex – Remix"          => :artist=>"KHEA"
  :title=>"Sincero"                              => :artist=>"Santi Celli"
  :title=>"Teco Teco"                            => :artist=>"Las Taradas"
  :title=>"Brindo"                               => :artist=>"Devendra Banhart"
  :title=>"Prélude in E minor, Op. 28, No. 4"    => :artist=>"Frédéric Chopin"
  :title=>"Tzuris Oy Vey"                        => :artist=>"Basil Kirchin"
  :title=>"Girls & Boys - 2012 Remaster"         => :artist=>"Blur"
  :title=>"What You Meant"                       => :artist=>"Franz Ferdinand"
  :title=>"Nosotros"                             => :artist=>"Babasónicos"
  :title=>"La Trampa es Ley"                     => :artist=>"LIT killah"
  :title=>"Sin Parangón"                         => :artist=>"El Kuelgue"
  :title=>"Like Glue"                          

In [28]:
first(sort(df, :ts),100)

Row,offline_timestamp,conn_country,master_metadata_album_album_name,username,spotify_track_uri,reason_start,ts,master_metadata_album_artist_name,offline,spotify_episode_uri,platform,reason_end,user_agent_decrypted,ms_played,master_metadata_track_name,ip_addr_decrypted,skipped,incognito_mode,episode_show_name,episode_name,shuffle
,Int64,String,Union…,String,Union…,String,DateTime,Union…,Bool,Union…,String,String,Union…,Int64,Union…,String,Union…,Bool,Union…,Union…,Bool
1,0,AR,Light and Heavy/The Best of...,11140152173,spotify:track:41TqRBgF4Ahyr4vGmZVk8b,appload,2014-05-11T23:58:43,Iron Butterfly,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,15629,In-A-Gadda-Da-Vida - Single Version,186.137.52.244,true,false,,,false
2,0,AR,Presente,11140152173,spotify:track:03PfjjVBo7pX8AOzfCJOhM,appload,2014-05-11T23:59:30,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,34080,Pena en mi corazon,186.137.52.244,true,false,,,false
3,0,AR,Presente,11140152173,spotify:track:3DBr8IWWZhH2F9KHmcBWN1,appload,2014-05-12T00:16:26,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",endplay,unknown,5572,La trufa y el sifon,186.137.52.244,true,false,,,false
4,0,AR,Presente,11140152173,spotify:track:2AtJX42EiltEH5FDTFYFyb,appload,2014-05-12T00:21:09,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,0,Caminante,186.137.52.244,true,false,,,false
5,0,AR,Presente,11140152173,spotify:track:5O4LBJJWwaso2p32Oz2q8Q,appload,2014-05-12T00:21:10,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,0,Pide piso,186.137.52.244,true,false,,,false
6,0,AR,Presente,11140152173,spotify:track:2AtJX42EiltEH5FDTFYFyb,appload,2014-05-12T00:23:17,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",backbtn,unknown,126037,Caminante,186.137.52.244,true,false,,,false
7,0,AR,Presente,11140152173,spotify:track:03PfjjVBo7pX8AOzfCJOhM,backbtn,2014-05-12T00:23:17,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",backbtn,unknown,0,Pena en mi corazon,186.137.52.244,true,false,,,false
8,0,AR,Presente,11140152173,spotify:track:4N5n65Zs60cSFRipzejIcZ,backbtn,2014-05-12T00:23:35,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",appload,unknown,18204,Nocturno,186.137.52.244,true,false,,,false
9,0,AR,Presente,11140152173,spotify:track:3DBr8IWWZhH2F9KHmcBWN1,appload,2014-05-12T00:30:00,Bajofondo,false,,"Android-tablet OS 4.4.2 API 19 (asus, Nexus 7)",trackdone,unknown,251813,La trufa y el sifon,186.137.52.244,false,false,,,false
